# Examining Racial Discrimination in the US Job Market

## Andy Pickering

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import math

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

I will consider only the race factor in this analysis

In [3]:
df = data[['race','call']]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 2 columns):
race    4870 non-null object
call    4870 non-null float32
dtypes: float32(1), object(1)
memory usage: 95.1+ KB


# (1) What test is appropriate for this problem? Does CLT apply?

- The question I am trying to answer is: Does the proportion of callbacks differ between the black and white resumes? The CLT applies; even though the population distributions are binomial, the sampling distribution of proportions will be approximately normal.

In [4]:
# compute proportion of callbacks for each group

df_w = df[df['race']=='w']
df_b = df[df['race']=='b']

# number of samples
N_w = len(df_w)
N_b = len(df_b)

# number that received call back
N_call_w = df_w.call.sum()
N_call_b = df_b.call.sum()

# proportion that received call back
P_w = N_call_w / N_w
P_b = N_call_b / N_b

[P_w,P_b]

[0.096509240246406572, 0.064476386036960986]

- About 9.7% of white resumes got call backs, compared to about 6.4% of black resumes. Now we need to determine if this difference is statistically significant.
- The null hypothesis is that there is no difference between the percent of black vs white- sounding resumes that receive callbacks.
- The alternative is that there is a significant difference.
- We can use a 2-sample t-test to determine if the two populations (black and white) are different. I will use the usual 5% threshold.

##  Compute margin of error, confidence interval, and p-value.

In [5]:
# construct 95% CI for (P_w-P_b)

# variances (from bernouli distribution)
var_w = P_w*(1-P_w) 
var_b = P_b*(1-P_b) 

# mean of sampling disriubtion of differences
mu_diff = P_w - P_b

# std of sampling distribution of differences
s_diff = math.sqrt( (var_w/N_w) + (var_b/N_b) )

# compute critical t_value for 95% CI
t_crit = stats.t.ppf(0.975,df=N_w-1)

# margin of error
d = t_crit * s_diff

# 95% confidence interval
CI = [mu_diff-d , mu_diff+d]
CI

[0.016770138495455986, 0.047295569923435185]

In [6]:
t_score = mu_diff/(s_diff)
t_score

4.1155504357300003

In [7]:
pval = stats.t.sf(t_score,df=N_w-1)*2
pval

3.991074585679982e-05

We can also use the built-in ttest from scipy.stats:

In [8]:
stats.ttest_ind(df_w.call,df_b.call,equal_var=False)

Ttest_indResult(statistic=4.1147052908617514, pvalue=3.9429415136459352e-05)

## Results
- The p-value from the t-test is very small, so we reject the null hypothesis and conclude that there is a significant difference in callbacks between white and black resumes.
- Or equivalently, the 95% confidence interval does not contain 0, so we conclude there is a significant difference in the means.

## Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?


In this analysis I only considered the factor of race; I would have to examine the other variables to determine which was the most immportant factor.